In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge,SGDRegressor

#from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
#from sklearn.datasets import load_digits
from sklearn.metrics import plot_confusion_matrix ,accuracy_score
from sklearn.naive_bayes import MultinomialNB,ComplementNB
from sklearn.model_selection import KFold,StratifiedKFold
import optuna
from sklearn.experimental import enable_hist_gradient_boosting


from sklearn.ensemble import HistGradientBoostingRegressor


In [3]:
train=pd.read_csv("../input/user-ui-preference/train.csv")
train.head()

In [9]:
train.describe()

In [4]:
train.isnull().sum()/len(train)*100

In [ ]:
train['State'].value_counts()

In [5]:
test=pd.read_csv('../input/user-ui-preference/test.csv')
test.head()

In [6]:
features=[x for x in train.columns if x not in ['CustomerID','City','Sign_up_date','Last_order_placed_date', 'Preferred_Theme']]
features

In [7]:
train['Gender']=train['Gender'].astype('category').cat.codes
train['State']=train['State'].astype('category').cat.codes
#train['Pedestrian_Crossing-Human_Control']=train['Pedestrian_Crossing-Human_Control'].astype('category').cat.codes
#train['Pedestrian_Crossing-Physical_Facilities']=train['Pedestrian_Crossing-Physical_Facilities'].astype('category').cat.codes
#train['Light_Conditions']=train['Light_Conditions'].astype('category').cat.codes
train.head()

In [11]:
train.Age = train.Age.fillna(value=train.Age.median(), axis=0)
train.No_of_orders_placed = train.No_of_orders_placed.fillna(value=train.No_of_orders_placed.median(), axis=0)
train['Kid’s_Clothing']=train['Kid’s_Clothing'].fillna(value=train['Kid’s_Clothing'].mean(), axis=0)
train['Home_&_Living']=train['Home_&_Living'].fillna(value=train['Home_&_Living'].mean(), axis=0)
train.head()

In [13]:
train.Preferred_Theme=train.Preferred_Theme.replace({'New_UI':0,'Old_UI':1})
train.head()

In [8]:
test['Gender']=test['Gender'].astype('category').cat.codes
test['State']=test['State'].astype('category').cat.codes
test.head()

In [12]:
test.Age = test.Age.fillna(value=test.Age.median(), axis=0)
test.No_of_orders_placed = test.No_of_orders_placed.fillna(value=test.No_of_orders_placed.median(), axis=0)
test['Kid’s_Clothing']=test['Kid’s_Clothing'].fillna(value=test['Kid’s_Clothing'].mean(), axis=0)
test['Home_&_Living']=test['Home_&_Living'].fillna(value=test['Home_&_Living'].mean(), axis=0)
test.head()

In [15]:
y=train.Preferred_Theme
X=train[features]
test_X=test[features]
test_X

In [19]:
n_folds = 5
test_roc_score = []
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)


for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    print('=============== Fold No:',fold+1,'===============')
    X_tr, X_tst = X.iloc[train_idx], X.iloc[test_idx]
    y_tr, y_tst = y.iloc[train_idx], y.iloc[test_idx]
    
    model = CatBoostClassifier()
    model.fit(X_tr, y_tr,eval_set=[(X_tst, y_tst)],verbose=500)
    testpred1 = model.predict_proba(X_tst)[:,1]
    test_roc_score.append(roc_auc_score(y_tst, testpred1))
    print("Test ROC AUC for model 1: %.4f"%(roc_auc_score(y_tst, testpred1)))
    
    pred = model.predict_proba(test_X)[:,1]
